### BentoML Example

# XGBoost League of legend Win prediction 

**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.

This is a BentoML Demo Project demonstrating how to train a League of Legend win prdiction model, and use BentoML to package and serve the model for building applictions.


Example notebook built based on https://slundberg.github.io/shap/notebooks/League%20of%20Legends%20Win%20Prediction%20with%20XGBoost.html

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=xgboost&ea=xgboost-league-of-legend-win-prediction&dt=xgboost-league-of-legend-win-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -q bentoml "numpy>=1.18.5" "xgboost==0.90" "scikit-learn>=0.23.0" "matplotlib>=3.2.2" "kaggle==1.5.6"

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as pl
from sklearn.model_selection import train_test_split

## Download Data

This notebook uses data from kaggle [paololol/league-of-legends-ranked-matches](https://www.kaggle.com/paololol/league-of-legends-ranked-matches)

You can set your Kaggle credential below and download the dataset automatically. The kaggle key can be created by going to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials, and fill it in the cell below.

Alternativelly, you can download it manually from [here](https://www.kaggle.com/paololol/league-of-legends-ranked-matches) and place unzip'd data in this folder.

In [ ]:
%%bash
export KAGGLE_USERNAME=
export KAGGLE_KEY=

if [ ! -f ./league-of-legends-ranked-matches.zip ]; then
    kaggle datasets download paololol/league-of-legends-ranked-matches
    unzip -n league-of-legends-ranked-matches.zip
fi

## Load data

In [4]:
# read in the data
matches = pd.read_csv("matches.csv")
participants = pd.read_csv("participants.csv")
stats1 = pd.read_csv("stats1.csv", low_memory=False)
stats2 = pd.read_csv("stats2.csv", low_memory=False)
stats = pd.concat([stats1,stats2])

# merge into a single DataFrame
a = pd.merge(participants, matches, left_on="matchid", right_on="id")
allstats_orig = pd.merge(a, stats, left_on="matchid", right_on="id")
allstats = allstats_orig.copy()

# drop games that lasted less than 10 minutes
allstats = allstats.loc[allstats["duration"] >= 10*60,:]

# Convert string-based categories to numeric values
cat_cols = ["role", "position", "version", "platformid"]
for c in cat_cols:
    allstats[c] = allstats[c].astype('category')
    allstats[c] = allstats[c].cat.codes
allstats["wardsbought"] = allstats["wardsbought"].astype(np.int32)

X = allstats.drop(["win"], axis=1)
y = allstats["win"]

# convert all features we want to consider as rates
rate_features = [
    "kills", "deaths", "assists", "killingsprees", "doublekills",
    "triplekills", "quadrakills", "pentakills", "legendarykills",
    "totdmgdealt", "magicdmgdealt", "physicaldmgdealt", "truedmgdealt",
    "totdmgtochamp", "magicdmgtochamp", "physdmgtochamp", "truedmgtochamp",
    "totheal", "totunitshealed", "dmgtoobj", "timecc", "totdmgtaken",
    "magicdmgtaken" , "physdmgtaken", "truedmgtaken", "goldearned", "goldspent",
    "totminionskilled", "neutralminionskilled", "ownjunglekills",
    "enemyjunglekills", "totcctimedealt", "pinksbought", "wardsbought",
    "wardsplaced", "wardskilled"
]
for feature_name in rate_features:
    X[feature_name] /= X["duration"] / 60 # per minute rate

# convert to fraction of game
X["longesttimespentliving"] /= X["duration"]

# define friendly names for the features
full_names = {
    "kills": "Kills per min.",
    "deaths": "Deaths per min.",
    "assists": "Assists per min.",
    "killingsprees": "Killing sprees per min.",
    "longesttimespentliving": "Longest time living as % of game",
    "doublekills": "Double kills per min.",
    "triplekills": "Triple kills per min.",
    "quadrakills": "Quadra kills per min.",
    "pentakills": "Penta kills per min.",
    "legendarykills": "Legendary kills per min.",
    "totdmgdealt": "Total damage dealt per min.",
    "magicdmgdealt": "Magic damage dealt per min.",
    "physicaldmgdealt": "Physical damage dealt per min.",
    "truedmgdealt": "True damage dealt per min.",
    "totdmgtochamp": "Total damage to champions per min.",
    "magicdmgtochamp": "Magic damage to champions per min.",
    "physdmgtochamp": "Physical damage to champions per min.",
    "truedmgtochamp": "True damage to champions per min.",
    "totheal": "Total healing per min.",
    "totunitshealed": "Total units healed per min.",
    "dmgtoobj": "Damage to objects per min.",
    "timecc": "Time spent with crown control per min.",
    "totdmgtaken": "Total damage taken per min.",
    "magicdmgtaken": "Magic damage taken per min.",
    "physdmgtaken": "Physical damage taken per min.",
    "truedmgtaken": "True damage taken per min.",
    "goldearned": "Gold earned per min.",
    "goldspent": "Gold spent per min.",
    "totminionskilled": "Total minions killed per min.",
    "neutralminionskilled": "Neutral minions killed per min.",
    "ownjunglekills": "Own jungle kills per min.",
    "enemyjunglekills": "Enemy jungle kills per min.",
    "totcctimedealt": "Total crown control time dealt per min.",
    "pinksbought": "Pink wards bought per min.",
    "wardsbought": "Wards bought per min.",
    "wardsplaced": "Wards placed per min.",
    "turretkills": "# of turret kills",
    "inhibkills": "# of inhibitor kills",
    "dmgtoturrets": "Damage to turrets"
}
feature_names = [full_names.get(n, n) for n in X.columns]
X.columns = feature_names

# create train/validation split
Xt, Xv, yt, yv = train_test_split(X,y, test_size=0.2, random_state=10)
dt = xgb.DMatrix(Xt, label=yt.values)
dv = xgb.DMatrix(Xv, label=yv.values)

## Train the XGBoost model


In [5]:
params = {
    "eta": 0.5,
    "max_depth": 4,
    "objective": "binary:logistic",
    "silent": 1,
    "base_score": np.mean(yt),
    "eval_metric": "logloss"
}
model = xgb.train(params, dt, 100, [(dt, "train"),(dv, "valid")], early_stopping_rounds=5, verbose_eval=10)

[15:31:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.54119	valid-logloss:0.54137
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 5 rounds.
[10]	train-logloss:0.34090	valid-logloss:0.34078
[20]	train-logloss:0.29880	valid-logloss:0.29892
[30]	train-logloss:0.27532	valid-logloss:0.27589
[40]	train-logloss:0.26305	valid-logloss:0.26410
[50]	train-logloss:0.25208	valid-logloss:0.25362
[60]	train-logloss:0.24394	valid-logloss:0.24587
[70]	train-logloss:0.23717	valid-logloss:0.23933
[80]	train-logloss:0.23103	valid-logloss:0.23352
[90]	train-logloss:0.22614	valid-logloss:0.22881
[99]	

In [6]:
Xt[:3]

,id_x,matchid,player,championid,ss1,ss2,role,position,id_y,gameid,...,Neutral minions killed per min.,Own jungle kills per min.,Enemy jungle kills per min.,Total crown control time dealt per min.,champlvl,Pink wards bought per min.,Wards bought per min.,Wards placed per min.,wardskilled,firstblood
1215555,1501034,150933,10,59,4,11,2,2,150933,3162804935,...,0.023086,0.023086,0.000000,7.572143,18,0.069257,0.0,0.831089,0.253944,0
1427835,1713614,172357,6,35,11,14,3,1,172357,3186087472,...,0.028262,0.028262,0.000000,10.937353,16,0.000000,0.0,0.197833,0.000000,0
1204118,1489597,149786,3,34,4,14,4,2,149786,3193266242,...,0.882817,0.693642,0.189175,11.192853,18,0.063058,0.0,0.567525,0.189175,0


In [7]:
model.predict(xgb.DMatrix(Xt[:3]))

array([0.35312122, 0.06715239, 0.04428952], dtype=float32)

## Create ML service with BentoML

In [8]:
%%writefile lol_win_predictions.py

from bentoml import api, env, BentoService, artifacts
from bentoml.frameworks.xgboost import XgboostModelArtifact
from bentoml.adapters import DataframeInput

import xgboost as xgb

@env(pip_packages=['xgboost'])
@artifacts([XgboostModelArtifact('model')])
class LeagueWinPrediction(BentoService):
    
    @api(input=DataframeInput(), batch=True)
    def predict(self, df):
        dmatrix = xgb.DMatrix(df)
        return self.artifacts.model.predict(dmatrix)

Overwriting lol_win_predictions.py


In [9]:
# 1) import the custom BentoService defined above
from lol_win_predictions import LeagueWinPrediction

# 2) `pack` it with required artifacts
bento_svc = LeagueWinPrediction()
bento_svc.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_svc.save()

[2020-09-22 15:36:13,398] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:36:13,853] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 15:36:13,856] WARNING - pip package requirement xgboost already exist
[2020-09-22 15:36:14,960] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py
set BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py to '0.9.0.pre+3.gcebf2015'
[2020-09-22 15:36:19,116] INFO - BentoService bundle 'LeagueWinPrediction:20200922153614_15BE7D' saved to: /Users/bozhaoyu/bentoml/repository/LeagueWinPrediction/20200922153614_15BE7D


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [10]:
!bentoml serve LeagueWinPrediction:latest

[2020-09-22 15:36:38,016] INFO - Getting latest version LeagueWinPrediction:20200922153614_15BE7D
[2020-09-22 15:36:38,017] INFO - Starting BentoML API server in development mode..
[2020-09-22 15:36:38,363] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:36:38,379] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 15:36:40,823] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 15:36:40,825] WARNING - pip package requirement xgboost already exist
 * Serving Flask app "LeagueWinPrediction" (lazy loading)
 * Environm

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve LeagueWinPrediction:latest --run-with-ngrok

### Make requeset to the REST server

*After navigate to the location of this notebook, copy and paste the following code to your terminal and run it to make request*

```bash
curl -i \
--request POST \
--header "Content-Type: text/csv" \
-d @test.csv \
localhost:5000/predict
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [11]:
!bentoml containerize LeagueWinPrediction:latest

[2020-09-22 15:38:10,877] INFO - Getting latest version LeagueWinPrediction:20200922153614_15BE7D
Found Bento: /Users/bozhaoyu/bentoml/repository/LeagueWinPrediction/20200922153614_15BE7D
[2020-09-22 15:38:10,922] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:38:10,939] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
Tag not specified, using tag parsed from BentoService: 'leaguewinprediction:20200922153614_15BE7D'
Building Docker image leaguewinprediction:20200922153614_15BE7D from LeagueWinPrediction:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/LeagueWinPrediction/20200922153614_15BE7D ['Docke

|Requirement already satisfied: grpcio<=1.27.2 in /opt/conda/lib/python3.7/site-packages (from bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (1.27.2)
\Requirement already satisfied: configparser in /opt/conda/lib/python3.7/site-packages (from bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (5.0.0)
|Collecting scipy
  Downloading scipy-1.5.2-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)
/Collecting pytz>=2011k
|Requirement already satisfied: itsdangerous>=0.24 in /opt/conda/lib/python3.7/site-packages (from flask->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (1.1.0)
\Requirement already satisfied: pyparsing>=2.0.2 in /opt/conda/lib/python3.7/site-packages (from packaging->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (2.4.7)
-Installing collected packages: scipy, xgboost, pytz, pandas
/Successfully installed pandas-0.24.2 pytz-2020.1 scipy-1.5.2 xgboost-1.2.0
- ---> cf3711f3c835
Step 8/15 : COPY . /bento
| ---> 3c517e745ea8
Step 9/15 : RUN if [ -d /bento/bu

-Requirement already satisfied, skipping upgrade: py-zipkin in /opt/conda/lib/python3.7/site-packages (from BentoML==0.9.0rc0+3.gcebf2015) (0.20.0)
/Requirement already satisfied, skipping upgrade: gunicorn in /opt/conda/lib/python3.7/site-packages (from BentoML==0.9.0rc0+3.gcebf2015) (20.0.4)
|Requirement already satisfied, skipping upgrade: yarl<2.0,>=1.0 in /opt/conda/lib/python3.7/site-packages (from aiohttp->BentoML==0.9.0rc0+3.gcebf2015) (1.5.1)
\Requirement already satisfied, skipping upgrade: idna<3,>=2.5 in /opt/conda/lib/python3.7/site-packages (from requests->BentoML==0.9.0rc0+3.gcebf2015) (2.10)
-  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.9.0rc0+3.gcebf2015-py3-none-any.whl size=3064091 sha256=8e787b2d7d257f7106558876f09d73902ba33b332788f147eefffacab48ccf04
  Stored in directory: /root/.cache/pip/wheels/a0/45/41/62152db705af4ff47e7a3d6abf6247986eef4aa1b94a58d3b9
Successfully built BentoML


-Installing collected packages: BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.9.0rc0
|    Uninstalling BentoML-0.9.0rc0:
/      Successfully uninstalled BentoML-0.9.0rc0
|Successfully installed BentoML-0.9.0rc0+3.gcebf2015
\ ---> e42b40e1aa88
Step 10/15 : ENV PORT 5000
- ---> Running in 5f759f6f786b
/ ---> 000eb066d01b
Step 11/15 : EXPOSE $PORT
 ---> Running in e1f3cbec65d9
| ---> be9e430ba73f
Step 12/15 : COPY docker-entrypoint.sh /usr/local/bin/
- ---> 286540998c4c
Step 13/15 : RUN chmod +x /usr/local/bin/docker-entrypoint.sh
 ---> Running in 5af2837f2622
| ---> 80d32914bfce
Step 14/15 : ENTRYPOINT [ "docker-entrypoint.sh" ]
\ ---> Running in 22ccfd449e4c
- ---> 83b5dbe3de6f
Step 15/15 : CMD ["bentoml", "serve-gunicorn", "/bento"]
 ---> Running in 81b057cf8222
/ ---> aba92d537770
Successfully built aba92d537770
Successfully tagged leaguewinprediction:20200922153614_15BE7D
Finished building leaguewinprediction:20200922153614_15BE7D fro

In [12]:
!docker run --rm -p 5000:5000 leaguewinprediction:20200922153614_15BE7D

[2020-09-22 22:40:28,656] INFO - Starting BentoML API server in production mode..
[2020-09-22 22:40:29,076] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-22 22:40:29 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-22 22:40:29 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-09-22 22:40:29 +0000] [1] [INFO] Using worker: sync
[2020-09-22 22:40:29 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-22 22:40:29 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-22 22:40:29 +0000] [14] [INFO] Booting worker with pid: 14
[2020-09-22 22:40:29,370] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-09-22 22:40:29,370] WARNING - Using BentoML not from official PyPI relea

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python: 

In [13]:
from bentoml import load

svc = load(saved_path)

print(svc.predict(Xt[:3]))

[2020-09-22 15:40:44,389] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 15:40:44,391] WARNING - Module `lol_win_predictions` already loaded, using existing imported module.
[2020-09-22 15:40:44,396] WARNING - pip package requirement pandas already exist
[2020-09-22 15:40:44,397] WARNING - pip package requirement xgboost already exist
[0.35312122 0.06715239 0.04428952]


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [ ]:
Xt[:3].to_csv('test.csv')

In [ ]:
!bentoml run LeagueWinPrediction:latest predict --input test.csv

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)